In [1]:
import pandas as pd
import numpy as np
import movie_stars
import warnings
from astropy.io import fits
warnings.filterwarnings("ignore")

time_to_integrate = np.linspace(0, -60, 50)


# apogee catalog to crossmatch with
df_apogee = pd.read_csv('/Users/cam/Downloads/apogee_dr17.csv')
age_min = 35
age_max = 60

In [2]:
radwave_progenitor_names = pd.read_csv('radcliffe_progenitor').group_name.values
split_proengenitor_names = pd.read_csv('split_progenitor').group_name.values

# Open Clusters

In [3]:
#df_open_clusters = pd.read_csv('/Users/cam/Downloads/cg_ocs_full.csv')
df_open_clusters = pd.read_csv('/Users/cam/Desktop/gaia_3d_plot_2/data/open_clusters.csv')
df_open_clusters = df_open_clusters.drop(columns=['radial_velocity'])

df_open_clusters = df_open_clusters.loc[(df_open_clusters['n_rv'] >= 10)]
df_open_clusters = df_open_clusters.loc[(np.abs(df_open_clusters.rv_best) < 100) & (
    df_open_clusters.rv_best.notnull()) & (df_open_clusters['parallax'] > 1.)]
#df_open_clusters = df_open_clusters.loc[df_open_clusters.rv_best_err <= 5.]
df_open_clusters = df_open_clusters.groupby(['cluster']).median().reset_index()
df_open_clusters = df_open_clusters.loc[df_open_clusters.age_myr <= age_max]


df1 = df_open_clusters.loc[df_open_clusters['cluster'].isin(radwave_progenitor_names)]
df2 = df_open_clusters.loc[df_open_clusters['cluster'].isin(split_proengenitor_names)]

open_clusters_key_dict = {'ra': 'ra',
                          'dec': 'dec',
                          'paralla': 'parallax',
                          'pmra': 'pmra',
                          'pmdec': 'pmdec',
                          'rv_best': 'radial_velocity',
                          'age_myr': 'age',
                          'cluster': 'group_name',
                          'n_rv' : 'n'
                          }
oc_marker_props = {'size': 5,
                   'color': 'cyan',
                   'opacity': 1.,
                   'symbol': 'age-based',
                   'line': dict(width=0.)
                   }


oc_group_1 = movie_stars.MovieStar(name='OCs Group 1',
                                            df=df_open_clusters,
                                            key_dict=open_clusters_key_dict,
                                            time_to_integrate=time_to_integrate,
                                            marker_properties=oc_marker_props
                                            )


In [4]:
oc_marker_props_2 = oc_marker_props.copy()
oc_marker_props_2['color'] = 'cyan'
oc_group_2 = movie_stars.MovieStar(name='OCs Group 2',
                                            df=df2,
                                            key_dict=open_clusters_key_dict,
                                            time_to_integrate=time_to_integrate,
                                            marker_properties=oc_marker_props_2
                                            )

# Sebastian Sco-Cen

In [5]:
#df_sc = pd.read_csv('/Users/cam/Downloads/sc_sebastian_apogee_and_sos.csv')
df_sc = pd.read_csv('/Users/cam/Downloads/sc_sebastian_full_new.csv')
df_sc = df_sc.loc[df_sc.n_rv > 10]

# creating movie class
sco_cen_key_dict = {'ra': 'ra',
                    'dec': 'dec',
                    'parallax': 'parallax',
                    'pmra': 'pmra',
                    'pmdec': 'pmdec',
                    'rv_chosen': 'radial_velocity',
                    'label': 'group_name',
                    'age': 'age'
                    }
sco_cen_marker_props = {'size': 5,
                        'opacity': 1.,
                        'symbol': 'age-based',
                        'color': 'magenta',
                        'line': dict(width=0.)
                        }

sco_cen = movie_stars.MovieStar(name='Sebastian Sco-Cen',
                                time_to_integrate=time_to_integrate,
                                df=df_sc,
                                key_dict=sco_cen_key_dict,
                                marker_properties=sco_cen_marker_props
                                )


sco_cen.marker_properties['symbol'] == 'age-based'


True

# Kerr+2021 Groups (besides Sco-Cen)

In [6]:
#df_kerr = pd.read_csv('/Users/cam/Downloads/kerr_xmatched.csv')
df_kerr = pd.read_csv('/Users/cam/Desktop/gaia_3d_plot_2/data/kerr_sos_edr3.csv')
df_kerr = df_kerr.rename(columns={'EOM' : 'eom'})
#(df_kerr.tlc != 22)
df_kerr = df_kerr.loc[(df_kerr.eom >= 0)]
df_kerr = pd.merge(left=df_kerr, right=df_apogee, how='left',
                   left_on='source_id', right_on='gaiaedr3_source_id')
df_kerr = df_kerr.loc[(df_kerr.parallax > 0.)]

df_kerr_apogee = df_kerr.loc[df_kerr['vhelio_avg'].notnull()]
df_kerr_apogee['rv_best'] = df_kerr_apogee['vhelio_avg']
df_kerr_apogee['rv_best_err'] = df_kerr_apogee['verr']

# df_kerr_gaia = df_kerr.loc[(df_kerr['vhelio_avg'].isnull()) & (df_kerr['radial_velocity'].notnull())]
# df_kerr_gaia['rv_best'] = df_kerr_gaia['radial_velocity']

df_kerr_sos = df_kerr.loc[(df_kerr['RVcor_merged'].notnull()) & (
    df_kerr['vhelio_avg'].isnull())]
df_kerr_sos['rv_best'] = df_kerr_sos['RVcor_merged']
df_kerr_sos['rv_best_err'] = df_kerr_sos['ERRVcor_merged']


df_kerr = pd.concat([df_kerr_apogee, df_kerr_sos])
df_kerr['rv_best_rel_err'] = np.abs(df_kerr['rv_best_err'].values/df_kerr['rv_best'].values)
df_kerr = df_kerr.loc[df_kerr['rv_best_rel_err'] < .3]

#df_kerr = df_kerr.drop(columns = ['radial_velocity'])

df_kerr = df_kerr.groupby(['eom']).median().reset_index()
df_kerr = df_kerr.loc[df_kerr.Age.notnull()]

kerr_key_dict = {'ra': 'ra',
                 'dec': 'dec',
                 'parallax': 'parallax',
                 'pmra': 'pmra',
                 'pmdec': 'pmdec',
                 'rv_best': 'radial_velocity',
                 'eom': 'group_name',
                 'Age': 'age'
                 }
kerr_marker_props = {'size': 5,
                     'opacity': 1.,
                     'symbol': 'age-based',
                     'color': '#4169E1',
                     'line': dict(width=0.)
                     }

kerr = movie_stars.MovieStar(name='Kerr+2021',
                             time_to_integrate=time_to_integrate,
                             df=df_kerr,
                             key_dict=kerr_key_dict,
                             marker_properties=kerr_marker_props)


# Nuria Groups

In [7]:
df_nuria = pd.read_csv('/Users/cam/Downloads/table_YLAs_equ_family2.csv')
df_nuria['age'] = np.ones(len(df_nuria))*35

nuria_key_dict = {'ra': 'ra',
                  'dec': 'dec',
                  'parallax': 'parallax',
                  'pmra': 'pmra',
                  'pmdec': 'pmdec',
                  'RV': 'radial_velocity',
                  'age': 'age',
                  'Group': 'group_name'}
nuria_marker_props = {'size': 5,
                      'color': 'orange',
                      'opacity': 1.,
                      'symbol': 'age-based',
                      'line': dict(width=0.)
                      }

nuria = movie_stars.MovieStar(name='Nuria Groups', time_to_integrate=time_to_integrate,
                              df=df_nuria, key_dict=nuria_key_dict, marker_properties=nuria_marker_props)


# Swiggum/Chen Orion

In [8]:
df_orion = pd.read_csv(
    '/Users/cam/Desktop/astro_research/orion/orion_full_groups_with_age.csv')
df_orion = df_orion.loc[df_orion.apogee2_radial_velocity.notnull()]
df_orion = df_orion.groupby(['label']).median().reset_index()
df_orion = df_orion.loc[df_orion.probability > .2]

orion_key_dict = {'ra': 'ra',
                  'dec': 'dec',
                  'parallax': 'parallax',
                  'pmra': 'pmra',
                  'pmdec': 'pmdec',
                  'apogee2_radial_velocity': 'radial_velocity',
                  'age': 'age',
                  'label': 'group_name'}
orion_marker_props = {'size': 9,
                      'color': '#8A2BE2',
                      'opacity': 1.,
                      'symbol': 'age-based',
                      'line': dict(width=0.)
                      }
orion = movie_stars.MovieStar(name='Swiggum/Chen Orion', time_to_integrate=time_to_integrate,
                              df=df_orion, key_dict=orion_key_dict, marker_properties=orion_marker_props)


In [9]:
# Hao clusters
#hdu_oc_hao = fits.open('/Users/cam/Downloads/asu-33.fit')
#df_oc_hao = pd.DataFrame(hdu_oc_hao[1].data)
df_oc_hao = pd.read_csv('/Users/cam/Desktop/gaia_3d_plot_2/data/hao_ocs_full.csv')
df_oc_hao = df_oc_hao.loc[df_oc_hao['n_rv'] >= 3]
df_oc_hao = df_oc_hao.groupby(['cluster']).median().reset_index()
df_oc_hao = df_oc_hao.drop(columns=['radial_velocity'])

df_oc_hao_young = df_oc_hao.loc[(df_oc_hao.age_myr < age_min)]
df_oc_hao_young = df_oc_hao_young.loc[df_oc_hao_young.rv_best.notnull()]

# df_oc_hao_older = df_oc_hao.loc[(df_oc_hao['age'].between(20.1, 100))]
# df_oc_hao_older = df_oc_hao_older.loc[df_oc_hao_older.rv_chosen.notnull()]

hao_key_dict = {'ra': 'ra',
                'dec': 'dec',
                'parallax': 'parallax',
                'pmra': 'pmra',
                'pmdec': 'pmdec',
                'rv_best': 'radial_velocity',
                'age_myr': 'age',
                'cluster': 'group_name'}
hao_marker_props = {'size': 5.,
                    'color': '#00CED1',
                    'opacity': 1.,
                    'symbol': 'age-based',
                    'line': dict(width=0.)
                    }
hao = movie_stars.MovieStar(name='Hao OCs (< 20 Myr)', time_to_integrate=time_to_integrate,
                            df=df_oc_hao_young, key_dict=hao_key_dict, marker_properties=hao_marker_props)

# hao_marker_props_older = hao_marker_props.copy()
# hao_marker_props_older['color'] = 'red'
# hao_older = movie_stars.MovieStar(name='Hao OCs (Age [Myr] < 80 Myr)', time_to_integrate=time_to_integrate,
#                             df=df_oc_hao_older, key_dict=hao_key_dict, marker_properties=hao_marker_props_older)


In [10]:
df_maria_rw = pd.read_csv('/Users/cam/Downloads/maria_rw_clusters.csv')
df_maria_rw = df_maria_rw.loc[(df_maria_rw.age_myr <= 60) & 
                        (df_maria_rw.rv_weighted.notnull()) & 
                        (df_maria_rw.rv_weighted != 0.) & 
                        (df_maria_rw.N_rv_weighted > 5) &
                        (df_maria_rw.rv_weighted.abs().between(-50,50))
                        ]
df_maria_rw = df_maria_rw.drop(columns = ['age','ra', 'de', 'pmra', 'radial_velocity'])

maria_clusters_key_dict = {'ra_dr3': 'ra',
                          'dec_dr3': 'dec',
                          'parallax_dr3': 'parallax',
                          'pmra_dr3': 'pmra',
                          'pmdec_dr3': 'pmdec',
                          'rv_weighted': 'radial_velocity',
                          'age_myr': 'age',
                          'cluster': 'group_name'
                          }

maria_marker_props = {'size': 8,
            'color': 'red',
            'opacity': 1.,
            'symbol': 'age-based',
            'line': dict(width=0.)
            }

maria_clusters_rw = movie_stars.MovieStar(name='Maria Clusters: RW',
                                            df=df_maria_rw,
                                            key_dict=maria_clusters_key_dict,
                                            time_to_integrate=time_to_integrate,
                                            marker_properties=maria_marker_props
                                            )

In [11]:
df_maria = pd.read_csv('/Users/cam/Downloads/maria_clusters.csv')
df_maria = df_maria.loc[(df_maria.age_myr <= 60) & 
                        (df_maria.rv_weighted.notnull()) & 
                        (df_maria.rv_weighted != 0.) & 
                        (df_maria.N_rv_weighted > 5) &
                        (df_maria.rv_weighted.abs().between(-50,50))
                        ]
df_maria = df_maria.drop(columns = ['age','ra', 'de', 'pmra', 'radial_velocity'])
# df_maria_young = df_maria.loc[df_maria.age_myr <= 25]


maria_marker_props = {'size': 3,
                   'color': 'gray',
                   'opacity': .7,
                   'symbol': 'age-based',
                   'line': dict(width=0.)
                   }


maria_clusters = movie_stars.MovieStar(name='Maria Clusters',
                                            df=df_maria,
                                            key_dict=maria_clusters_key_dict,
                                            time_to_integrate=time_to_integrate,
                                            marker_properties=maria_marker_props
                                            )

In [12]:
# make movie
movie_save_path = '/Users/cam/Desktop/astro_research/radcliffe/movies/galactic_motions_movie_2.html'
#data_list = [oc_group_1, oc_group_2, kerr, nuria, orion, sco_cen, maria_clusters]
data_list = [maria_clusters, maria_clusters_rw, sco_cen, kerr, orion]

annotations = [dict(x=-272, y=120, z=50, text='Radcliffe Wave (t = 0)', xanchor='center', opacity=.8, font=dict(size=10, color='red'), showarrow=False),
               dict(x=235, y=77, z=44, text='The Split (t = 0)', xanchor='center',
                    opacity=.8, font=dict(size=10), showarrow=False),
               dict(x=2144, y=-5768, z=60, text='Compact Group? (t = -26.5)',
                    xanchor='center', opacity=1., font=dict(size=8), showarrow=True),
               dict(x=6185, y=-7507, z=20, text='Similar gas-origins for Sco-Cen and OCs? (t = -48.5)',
                    xanchor='center', opacity=1., font=dict(size=8), showarrow=True),
               dict(x=6929, y=-7574, z=0, text='Similar gas-origins for Sco-Cen and OCs? (t = -48.5)',
                    xanchor='center', opacity=1., font=dict(size=8), showarrow=True),
               dict(x=379, y=-3096, z=-94, text='Sun passes through Orion (t=-13.3)',
                    xanchor='center', opacity=1., font=dict(size=8), showarrow=True)
               ]

movie = movie_stars.Movie(
    movie_stars=data_list, time=time_to_integrate, movie_save_path=movie_save_path,
    center_star='Sun', annotations=None, xyz_ranges=[5000, 5000, 400]
)
movie.make_movie()


Movie Created!


In [13]:
maria_clusters.df_integrated.to_csv('/Users/cam/Downloads/maria_clusters_int.csv', index=False)
maria_clusters_rw.df_integrated.to_csv('/Users/cam/Downloads/maria_clusters_rw_int.csv', index=False)